In [ ]:
# default_exp __init__
# hide
import unittest
from unittest import mock
import os
TESTCASE = unittest.TestCase()

In [ ]:
#export
_FNAME='init'

In [ ]:
#export
import datetime
import pandas as pd

import holidays
import logging
logger = logging.getLogger()

from pydantic import BaseModel, validator
from typing import List, Optional, Union
from collections import OrderedDict

# Scheduling

A nice interface to write a building schedule

In [ ]:
import yaml
pdt = pd.to_datetime

## 🕒 Period
A period is a uninterupted stretch of time that is all the same status.  
E.g. `{'start': "9:00", 'end': "15:00", 'status': 'occupied'}` means the building is occupied from 9am-5pm.

In [ ]:
#export 
class Period(BaseModel):
    start: datetime.time
    end: datetime.time
    status: Optional[str] = 'occupied'
    def __repr__(self):
        return '<{stat} Period {s}-{e}>'.format(stat=self.status, s=self.start, e=self.end)
    def __str__(self):
        return self.__repr__()
    def during(self, ts) -> bool:
        '''
        ts: a timestamp. Can be a datetime or any other obj with a .time() method.
        If no .time method exists, ts can also be any object that can be compared with datetime.time objects
        '''
        try:
            tod = ts.time()
        except AttributeError:
            tod=ts
        #end needs to be <= to allow for a 24 hour day
        return self.start <= tod <= self.end   
    

In [ ]:
def test_opening():
    per = Period(**{'start': "6:00", 'end': "18:00"})
    TESTCASE.assertTrue(per.during( pdt('2021-01-01 6:01')  ))
    TESTCASE.assertFalse(per.during( pdt('2021-01-01 20:00')  ))  

    alwayson = Period(start="0:00", end="0:00", status='some other status')
    TESTCASE.assertTrue(alwayson.during( pdt('2021-01-01 00:00')))
    print('✅ Passing')
    
    
test_opening()

✅ Passing


## ☀️ Day 🌗 
A day is made up of zero or more periods.  These periods can have different statuses if you want.  You can then ask the day whether a certain time of day falls within any period of a given status.  
> e.g. is 10:32 during occupied hours?

In [ ]:
#export
class Day(BaseModel):
    '''
    A type of day that contains the periods listed
    If a single period is given, that is the only period for the dat
    '''
    periods: List[Period]=[]
    name: Optional[str]=None
        
    def __repr__(self):
        name = self.name or ''
        return '<{n} Day: {periods}>'.format(n=name, periods=self.periods)
    def __str__(self):
        return self.__repr__()    
        
    def within_period(self, ts, status='occupied') -> bool:
        checks = [p.during(ts) for p in self.periods if p.status==status]
        return sum(checks) >= 1
    
    def get_all_status(self):
        statuses = [period.status for period in self.periods]
        return list(dict.fromkeys(statuses))
    
    def get_status(self, ts):
        '''
        Returns the status of the time ts given or None if it is not within any of the periods.
        '''
        for status in self.get_all_status():
            if self.within_period(ts, status=status):
                return status
    
        

In [ ]:
day_config = yaml.safe_load('''
    periods:
        - start: '9:00'
          end: '17:00'
          status: occupied
    name: workday
''')

closed_for_lunch = yaml.safe_load('''
    periods:
        - start: '9:00'
          end: '12:00'
          status: occupied
        - start: '13:00'
          end: '20:00'
          status: occupied
    name: split_workday
''')

def test_day():
    day = Day(**day_config)
    #too early
    TESTCASE.assertFalse(day.within_period( pdt('2021-01-01 8:00')))
    #1 min into the workday
    TESTCASE.assertTrue(day.within_period( pdt('2021-01-01 9:01')))
    #status of the period does not match the passed in status
    TESTCASE.assertFalse(day.within_period( pdt('2021-01-01 9:01'), status='otherstatus')) 
    
    cfl = Day(**closed_for_lunch)
    #too early
    TESTCASE.assertFalse(cfl.within_period( pdt('2021-01-01 8:00')))
    TESTCASE.assertIsNone(cfl.get_status( pdt('2021-01-01 8:00')))
    
    #1 min into the workday
    TESTCASE.assertTrue(cfl.within_period( pdt('2021-01-01 9:01')))
    #lunch
    TESTCASE.assertFalse(cfl.within_period( pdt('2021-01-01 12:01')))
    #1 min into the workday
    TESTCASE.assertTrue(cfl.within_period( pdt('2021-01-01 19:59')))
    TESTCASE.assertEqual(cfl.get_status( pdt('2021-01-01 19:59')), 'occupied')
    
    two_status_day = yaml.safe_load('''
        periods:
            - start: '9:00'
              end: '17:00'
              status: occupied
            - start: '17:00'
              end: '19:00'
              status: cleaning
        name: workday
    ''')   
    daystats = Day(**two_status_day)
    TESTCASE.assertEqual(set(daystats.get_all_status()), set(['occupied', 'cleaning']))    
    print('✅ Passing')    
    
test_day()

✅ Passing


## 🤔 Logic
After defining different types of days (e.g. workdays, offdays, etc) we need logic to see whether a certain date corresponds to that type of day.

Logic functions must accept a `pd.Timestamp` object `ts` as the first argument.  Other optional keywords can be given.  These will be fed in from the `kwargs` in the config for the `Logic` object defined below.

In [ ]:
#export

def always(ts):
    return True

def is_weekend(ts):
    '''
    Returns True if the given ts is Saturday or Sunday
    Pandas defines the day of the week with Monday=0, Sunday=6.
    '''
    dayofweek = ts.dayofweek
    return dayofweek >= 5

def is_weekday(ts):
    '''
    Returns True if the given ts is a weekday (Monday-Friday)
    '''
    
    dayofweek = ts.dayofweek
    return dayofweek <= 4

def get_named_weekday(dayname) -> int:
    '''
    Returns the number day of the week given the name
    '''
    days = {'MONDAY':0,
           'TUESDAY':1,
           'WEDNESDAY':2,
            'THURSDAY': 3,
            'FRIDAY': 4,
            'SATURDAY': 5,
            'SUNDAY': 6
           }
    return days[dayname.upper()]

def specific_weekday(ts, dayname:str):
    '''
    Returns True if the ts is that specific day of the week.
    e.g. 
    specific_weekday(pdt('2021-07-05'), 'Monday') --> True
    '''
    return ts.dayofweek == get_named_weekday(dayname)

def year_of(ts):
    year = ts.year
    return '{y}-01-01'.format(y=year), '{y}-12-31'.format(y=year)

def is_holiday(ts, holiday_calendar='US'):
    _holiday_calendars = {'US': holidays.UnitedStates()}
    hcal = _holiday_calendars.get(holiday_calendar, holiday_calendar)
    return ts.date() in hcal

def is_month(ts, months):
    return ts.month in months

def make_year_specific_date(year, month=None, day=None):
    if month and day:
        return pd.to_datetime('{y}-{m}-{d}'.format(y=year, m=month, d=day))

def before(ts, month:int, day:int) -> bool:
    '''
    Teturns True if the ts is earlier in the year than date.
   '''
    return ts < make_year_specific_date(ts.year, month, day)

def after(ts, month, day):
    return ts > make_year_specific_date(ts.year, month, day)

In [ ]:
def test_logic_functions():
    TESTCASE.assertTrue(is_month(pdt('2021-11-01'), [11, 12]))
    #because july 4th 2021 was a sunday
    TESTCASE.assertTrue(is_weekend(pdt('2021-07-04')))
    #july 5th was a holiday in the US on 2021, because july 4th was a sunday
    TESTCASE.assertTrue(is_holiday(pdt('2021-07-05')))
    feb1 = pdt('2021-02-01')

    TESTCASE.assertTrue(before(feb1, month=3, day=1))
    TESTCASE.assertFalse(after(feb1, month=3, day=1))

    TESTCASE.assertFalse(before(feb1, month=1, day=31))
    TESTCASE.assertTrue(after(feb1, month=1, day=31))
    TESTCASE.assertTrue(specific_weekday(pdt('2021-07-05'), 'Monday'))

    print('✅ Passing')
    
test_logic_functions()

✅ Passing


In [ ]:
#export
NAMED_FUNCTIONS = {'weekends': is_weekend,
                   'weekend': is_weekend,
                   'weekdays': is_weekday,
                   'weekday': is_weekday,
                   'holidays': is_holiday,
                   'holiday': is_holiday,         
                   'always': always,
                   'months': is_month,
                   'before': before,
                   'after': after,
                   
        }

In [ ]:
#export
class Logic(BaseModel):
    include: Optional[str]
    exclude: Optional[str]
    kwargs: Optional[dict]={}
        
    @validator('include')
    @validator('exclude')    
    def func_must_be_a_known_name(cls, logic_string):
        if logic_string not in NAMED_FUNCTIONS:
            raise ValueError("logic must be one of {}".format(NAMED_FUNCTIONS.keys()))
        return logic_string
    
    def evaluate(self, ts) -> bool:
        '''
        Returns True if the function named in self.func evaluates to true
        '''
        func_name = self.include or self.exclude
        func = NAMED_FUNCTIONS[func_name]
        result = func(ts, **self.kwargs)
        if self.exclude:
            result = not result
        return result
    

In [ ]:
def test_logic():
    with TESTCASE.assertRaises(ValueError):
        log = Logic(include='doesnotexist')

    #always will give you true for any value
    TESTCASE.assertTrue(Logic(include='always').evaluate(0))

    halloween = yaml.safe_load('''
    include: months
    kwargs:
        months: 
            - 10
    ''')
    TESTCASE.assertTrue(Logic(**halloween).evaluate(pdt('2020-10-31')))
    
    closed_on_holidays = yaml.safe_load('''
    exclude: holidays
    kwargs:
        holiday_calendar: US
    ''')
    TESTCASE.assertFalse(Logic(**closed_on_holidays).evaluate(pdt('2020-12-25')))
    
test_logic()

## Day Types

Attach the built-in logic to a day

In [ ]:
#export

class DayType(BaseModel):
    name: Optional[str]=None
    logic: List[Logic] = []
    day:Day
        
    def evaluate_logic(self, ts) -> bool:
        '''
        Returns True if ALL the functions in self.funcs evaluate to True.
        Returns False otherwise.
        Returns False if the list of self.funcs is empty.
        '''
        result = False
        #a Logics with no funcs always returns False
        for logic in self.logic:
            result = logic.evaluate(ts)
            if result is False:
                return False
        return result        
            
    def evaluate(self, ts) -> Union[Day, None]:
        '''
        Returns the day object if it evaluates to true for the given timestamp ts.
        Otherwise, returns None
        '''
        if self.evaluate_logic(ts):
            return self.day
        

In [ ]:
def test_daytype():
    workday = yaml.safe_load('''
    name: workday-type
    day:
        periods:
            - start: '9:00'
              end: '17:00'
              status: occupied
        name: workday
    logic: 
        - include: weekday
    ''')    
    dt = DayType(**workday)
    TESTCASE.assertIs(dt.evaluate(pdt('2021-07-07')), dt.day)
    TESTCASE.assertIsNone(dt.evaluate(pdt('2021-07-10')))
test_daytype()

In [ ]:
   
def test_logics():
    october_weekdays = yaml.safe_load('''
    day: {}
    logic:
        - include: months
          kwargs:
            months:
                - 10
        - include: weekday
    ''')
    octs = DayType(**october_weekdays)
    
    #true for october weekday
    TESTCASE.assertTrue(octs.evaluate(pdt('2020-10-30')))
    #halloween was a saturday        
    TESTCASE.assertFalse(octs.evaluate(pdt('2020-10-31')))
    TESTCASE.assertFalse(octs.evaluate_logic(pdt('2020-10-31')))    
    #false for any day in november
    TESTCASE.assertFalse(octs.evaluate(pdt('2020-11-10')))    

    # order doesnt matter since the logic is AND
    october_weekdays_reversed = yaml.safe_load('''
    day: {}
    logic:
        - include: weekday    
        - include: months
          kwargs:
            months:
                - 10
    ''')
    octs_rev = DayType(**october_weekdays_reversed)
    #true for october weekday
    TESTCASE.assertTrue(octs_rev.evaluate(pdt('2020-10-30')))
    #halloween was a saturday        
    TESTCASE.assertFalse(octs_rev.evaluate(pdt('2020-10-31')))    
    #false for any day in november
    TESTCASE.assertFalse(octs_rev.evaluate(pdt('2020-11-10')))
    
    weekdays_no_holidays = yaml.safe_load('''
    day: {}
    logic:
        - include: weekdays   
        - exclude: holidays
    ''')
    #july 5th 2021 was a monday observation of independence day
    wnh = DayType(**weekdays_no_holidays)
    TESTCASE.assertFalse(wnh.evaluate(pdt('2021-07-05')))
    TESTCASE.assertTrue(wnh.evaluate(pdt('2021-07-06')))
    
    #empty lists always evaluate to false
    TESTCASE.assertFalse(DayType(day={}, funcs=[]).evaluate(0))
    print('✅ Passing')
    
    
test_logics()

✅ Passing


## 📅 Schedule

In [ ]:
#export 
class IncompleteSchedule(Exception):
    pass

class Schedule(BaseModel):
    daytypes:List[DayType] = []        

    def find_relevant_day(self, ts):
        '''
        Returns the first day for which the logic func(**kwargs) evaluates True
        '''
        for daytype in self.daytypes:
            day = daytype.evaluate(ts)
            if day:
                return day

    def is_occupied(self, ts):
        return self.check_status(ts, status='occupied')
    
    def check_status(self, ts, status):
        day = self.find_relevant_day(ts)
        if day:
            return day.within_period(ts, status=status)
        else:
            return False
        
    def get_status(self, ts):
        day = self.find_relevant_day(ts)
        return day.get_status(ts)
        

In [ ]:
def test_schedule():
    worksched = yaml.safe_load('''
daytypes: 
      - name: workday-type
        day:
            periods:
                - start: '6:30'
                  end: '18:30'
                  status: occupied
            name: workday
        logic:
            - include: weekdays
            - exclude: holidays
            - exclude: months
              kwargs:
                  months: 
                      - 1
      - name: half-day-janurary
        day: 
            periods:
                - start: '6:30'
                  end: '12:00'
                  status: 'occupied'
            name: halfday
        logic: 
            - include: weekdays
            - include: months
              kwargs:
                  months:
                      - 1
    ''')
    sched = Schedule(**worksched)    
    jul6 = pdt('2021-07-06 01:00')
    #jul 6 picks up the workday-type of day
    TESTCASE.assertIs(sched.find_relevant_day(jul6), sched.daytypes[0].day)
    #but it evaluates to unoccupied at 1am
    TESTCASE.assertFalse(sched.is_occupied(jul6))
    jul6noon = pdt('2021-07-06 12:00')
    TESTCASE.assertIs(sched.find_relevant_day(jul6noon), sched.daytypes[0].day)
    #however it is occupied at noon
    TESTCASE.assertTrue(sched.is_occupied(jul6noon))
    
    #they work a half day all weekdays in jan.
    jan1 = pdt('2021-01-01 08:00')
    TESTCASE.assertIs(sched.find_relevant_day(jan1), sched.daytypes[1].day)
    TESTCASE.assertTrue(sched.is_occupied(jan1))
    TESTCASE.assertEqual(sched.get_status(jan1), 'occupied')
    
    #but jan2 is a saturday
    TESTCASE.assertFalse(sched.is_occupied(pdt('2021-01-02 08:00')))
    print('✅ Passing')
    
test_schedule()

✅ Passing


In [ ]:
from nbdev.export import notebook2script
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
notebook2script(_nbpath)

Converted init.ipynb.
